<a href="https://colab.research.google.com/github/Poornima1994/Poorni_repo/blob/main/Moview_Review_Sentiment_Analysis_Clasiification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import re

In [18]:
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score,confusion_matrix,recall_score,f1_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [19]:
import warnings
warnings.filterwarnings('ignore')

In [61]:
import pandas as pd
header_list = ["Review", "Sentiment"]
imdb_df = pd.read_csv('/content/IMDB Dataset.csv')


In [94]:
def remove_html_tags(text):
    cleaner = re.compile('<.*?>')
    cleantext = re.sub(cleaner, ' ', text)
    return cleantext

def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    text=re.sub("\s\s+" , " ", text)
    text=text.lower()
    return text

def preprocess_review(text):
    clean_text = remove_html_tags(text);
    clean_text = remove_between_square_brackets(clean_text);
    clean_text = remove_special_characters(clean_text)
    return clean_text;

In [95]:
imdb_df.iloc[666]['review']

'This was a fine example of how an interesting film can be made without using big stars and big effects. Just tell a true story about the struggles of two African American women over a turbulent century.<br /><br />This movie challenges us all to look at our own personal prejudices and see that people are people, not white, black, etc.<br /><br />Good movie with a good message.'

In [97]:
preprocess_review(imdb_df.iloc[666]['review'])

'this was a fine example of how an interesting film can be made without using big stars and big effects just tell a true story about the struggles of two african american women over a turbulent century this movie challenges us all to look at our own personal prejudices and see that people are people not white black etc good movie with a good message'

In [98]:
imdb_df['review'] = imdb_df["review"].apply(lambda x: preprocess_review(x))

In [99]:
imdb_df['new_sentiment'] = imdb_df['sentiment'] == "positive"
imdb_df["new_sentiment"] = imdb_df["new_sentiment"].astype("uint8")
imdb_df = imdb_df.drop("sentiment", axis=1)
imdb_df = imdb_df.rename(columns={"new_sentiment": "sentiment"})

In [100]:
imdb_df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production the filming tech...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically theres a family where a little boy j...,0
4,petter matteis love in the time of money is a ...,1


Splitting data

In [102]:

X = imdb_df[:5000]['review']
y = imdb_df[:5000]['sentiment']

In [103]:
from sklearn.model_selection import train_test_split
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, random_state=42, test_size=0.1)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=42, test_size=0.1)

In [104]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((4050,), (4050,), (450,), (450,), (500,), (500,))

In [105]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [106]:
for example, label in train_dataset.take(1):
    print('text: ', example.numpy())
    print('label: ', label.numpy())

text:  b'star rating saturday night friday night friday morning sunday night monday morning one time heroin addict frankie machine frank sinatra gets out of prison to his bumbling jailbird partner sparrow arnold stang needy cripple of a wife zosch eleanor parker and bit on the side molly kim novak hes trying to make it big as a drummer in a band but until his big break comes along hes stuck doing the only other thing he was any good at other than being a junkie dealing cards in high stakes games and try as he might even prison hasnt cured him of his addiction to the devils drug causing him to lie to and deceive all those around him and driving him to desperate measures to feed his habit his yearning to come off it is his only motivation towards a happy ending when people think of frank sinatra they generally think of classic high pitched songs like under my skin new york new york and it had to be you but lest anyone forget he was actually a renowned actor too and if his performance in 

In [107]:
BATCH_SIZE = 32

In [108]:
train_dataset = train_dataset.shuffle(5000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(500).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(500).prefetch(tf.data.AUTOTUNE)


In [109]:
for example, label in train_dataset.take(1):
    print('texts: ', example.numpy()[:3])
    print()
    print('labels: ', label.numpy()[:3])

texts:  [b'the only good part about this film is the beautiful scenery this movie was long and boring the minister should have retired from the pulpit the time his son paul strayed from the teachings he proclaimed how many times can his boys take the lords name in vain in this film being from a presbyterian background it doesnt fit i wished paul was swept down the river without a boat at the very beginning to spare us the silly smirkish selfish story of paul brad pitt so norm becomes a teacher and paul becomes a compulsive gambler who norm wants to rescue but doesntso what its very uninteresting we see the prejudiced whites being stood up to by paul because of his native american girl that was the only part that had some interest and maybe could have been developed into a real wild western what we only see is a sleepy town where the two ministers sons have nothing to do but 1 norm chase a lame girlfriend and deal with her family and 2paul make up dumb stories at the newspaper shop whil

In [110]:
VOCAB_SIZE=2000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [111]:
vocab = np.array(encoder.get_vocabulary())
vocab[:50]

array(['', '[UNK]', 'the', 'a', 'and', 'of', 'to', 'is', 'in', 'it',
       'this', 'i', 'that', 'was', 'as', 'with', 'movie', 'for', 'but',
       'film', 'on', 'you', 'not', 'are', 'have', 'his', 'be', 'one',
       'he', 'its', 'at', 'all', 'by', 'an', 'from', 'they', 'so', 'who',
       'like', 'just', 'about', 'or', 'if', 'her', 'out', 'has', 'some',
       'what', 'there', 'good'], dtype='<U14')

In [112]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

array([[  2,  68,  49, ...,   0,   0,   0],
       [428, 754,   9, ...,   0,   0,   0],
       [142, 253, 778, ...,   0,   0,   0]])

In [113]:
for n in range(1):
    print("Original: ", example[n].numpy())
    print("\nRound-trip: ", " ".join(vocab[encoded_example[n]]))
    print()

Original:  b'the only good part about this film is the beautiful scenery this movie was long and boring the minister should have retired from the pulpit the time his son paul strayed from the teachings he proclaimed how many times can his boys take the lords name in vain in this film being from a presbyterian background it doesnt fit i wished paul was swept down the river without a boat at the very beginning to spare us the silly smirkish selfish story of paul brad pitt so norm becomes a teacher and paul becomes a compulsive gambler who norm wants to rescue but doesntso what its very uninteresting we see the prejudiced whites being stood up to by paul because of his native american girl that was the only part that had some interest and maybe could have been developed into a real wild western what we only see is a sleepy town where the two ministers sons have nothing to do but 1 norm chase a lame girlfriend and deal with her family and 2paul make up dumb stories at the newspaper shop wh

Model creation

In [114]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=128,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [115]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [116]:
checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True)

In [118]:
history = model.fit(train_dataset, epochs=5,
                    validation_data=val_dataset, callbacks=[early_stop, model_checkpoint_callback])

Epoch 1/5
127/127 [==============================] - 433s 3s/step - loss: 0.6926 - accuracy: 0.5289 - val_loss: 0.6910 - val_accuracy: 0.5267
Epoch 2/5
127/127 [==============================] - 428s 3s/step - loss: 0.6069 - accuracy: 0.6854 - val_loss: 0.4302 - val_accuracy: 0.8000
Epoch 3/5
127/127 [==============================] - 427s 3s/step - loss: 0.3613 - accuracy: 0.8477 - val_loss: 0.3826 - val_accuracy: 0.8444
Epoch 4/5
127/127 [==============================] - 433s 3s/step - loss: 0.2919 - accuracy: 0.8827 - val_loss: 0.3570 - val_accuracy: 0.8533
Epoch 5/5
127/127 [==============================] - 435s 3s/step - loss: 0.2453 - accuracy: 0.9094 - val_loss: 0.3337 - val_accuracy: 0.8489


Predicting sample reviews

In [119]:
examples_pos = [
  "The movie was great, i would recommend this movie to my family, the storyline is amazing",
 ]

result=model.predict(examples_pos)
print("positive" if (result>0.5) else "negative" )

positive


In [120]:
examples_neg = [
  "The movie was terrible, dont watch this movie, dont waste your time watching these movies!"
]

result1=model.predict(examples_neg)
print("positive" if (result1>0.5) else "negative" )

negative
